In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import numpy as np
import gstlearn as gl
import gstlearn.plot as gp
import urllib.request

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.csv'
filecsv, head = urllib.request.urlretrieve(url)
datcsv = pd.read_csv(filecsv)
datcsv

In [ ]:
# Replace missing values and convert to numeric
datcsv["January_temp"].replace("MISS", np.nan, inplace=True)
datcsv = datcsv.astype("float64")

In [ ]:
data = gl.Db()
data[list(datcsv.columns)] = datcsv
data.setLocators(["Longitude","Latitude"], gl.ELoc.X)
data.setLocator("January_temp", gl.ELoc.Z)
data

In [ ]:
gp.plot(data)

In [ ]:
grid = gl.DbGrid.createCoveringDb(data, [80,130], margin=[50,50])
grid

In [ ]:
gp.plot(grid)
gp.symbol(data)

In [ ]:
nmini = 1
nmaxi = 3
radius = 30
nsect = 1
neigh = gl.NeighMoving.create(False, nmaxi, radius, nmini, nsect)
neigh

In [ ]:
node = 4511
neigh.attach(data, grid)
ranks = neigh.select(node)
ranks # Only rank 143

In [ ]:
def show_neigh(data, grid, neigh, node, ranks):
  ax = data.plot()
  if (len(ranks) > 0):
    dataSel = data.clone()
    dataSel.addSelectionByRanks(ranks)
    ax = dataSel.plot(color='blue')
  ax.neigh(neigh, grid, node, flagCell=True)
  ax.decoration("Standard Neighborhood")
    
show_neigh(data, grid, neigh, node, ranks)

In [ ]:
# OPTION 1
class BiTargetCheckMaxRadius(gl.BiTargetCheckDistance): # OK but override doesn't work
# OPTION 2
#class BiTargetCheckMaxRadius(gl.ABiTargetCheck): # NOK : No constructor defined - class is abstract
    def __init__(self, r):
        # If OPTION 1, puting the radius in __init__ below leads to select the good rank
        super(BiTargetCheckMaxRadius,self).__init__()
        self.r = r
    def isOK(self,st1,st2):
        print("isOk")
        return st1.getDistance(st2) < self.r
    def toString(self,strfmt):
        return "coucou"
    def clone(self):
        return self

In [ ]:
nmini = 1
nmaxi = 3
neigh2 = gl.NeighMoving.create(False, nmaxi=nmaxi, nmini=nmini)
btc = BiTargetCheckMaxRadius(30)
neigh2.addBiTargetCheck(btc)
neigh2.display() # This should display the additionnal BiTargetCheck (cloning issue?)

In [ ]:
type(neigh2.getBipts(0)) # It exists !

In [ ]:
btc.display() # "coucou" should be displayed! => NOK :-(

In [ ]:
node = 4511
neigh2.attach(data, grid)
ranks = neigh2.select(node)
ranks # Wrong ranks

In [ ]:
show_neigh(data, grid, neigh2, node, ranks)